In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,Dense,MaxPool2D,Flatten
from tensorflow.keras.regularizers import l2
import os
from os import listdir
import gdown
from zipfile import ZipFile
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer

In [2]:
from tensorflow.keras.utils import img_to_array
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            img_gan = cv2.resize(image, (32,32))
            return img_to_array(img_gan)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [3]:
def extract_plant_imgs(directory_root):
    image_list, label_list = [], []
    try:
        print("[INFO] Loading images ...")
        root_dir = listdir(directory_root)
        #print(root_dir)
        for plant_folder in root_dir :
            # remove .DS_Store from list
            if plant_folder == ".DS_Store" :
                root_dir.remove(directory)


        for plant_folder in root_dir :
    #         print("on plant folder")
            plant_image_list = listdir(f"{directory_root}/{plant_folder}")



            for single_image in plant_image_list :

    #             print("in plant folder")
                if single_image == ".DS_Store" :
                    plant_image_list.remove(single_image)

            for image in plant_image_list[:1000]:
    #             print("getting img path")
                image_directory = f"{directory_root}/{plant_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True or image_directory.endswith(".png") == True or image_directory.endswith(".PNG") == True:
    #                 print("checking if img correct format")
                    gan_img = convert_image_to_array(image_directory)
                    image_list.append(gan_img)
                    label_list.append(plant_folder)

        print("[INFO] Image loading completed")
        return image_list, label_list
    except Exception as e:
        print(f"Error : {e}")

In [4]:
# enhanced_data = 'Enhanced_tomato_DCGAN'

In [5]:
# enhanced_image_list , enhanced_label_list = extract_plant_imgs(enhanced_data)
orignal_img_list, orignal_label_list = extract_plant_imgs('Orignal_plant')

[INFO] Loading images ...
[INFO] Image loading completed


In [113]:
GAN_image_list, GAN_label_list = extract_plant_imgs('BiGANxSRGAN_plant')

[INFO] Loading images ...
[INFO] Image loading completed


In [114]:
normalized_image_list_GAN = np.array(GAN_image_list, dtype = np.float16) / 255.0
normalized_image_list_orignal = np.array(orignal_img_list, dtype = np.float16) / 255.0

In [115]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
image_labels_GAN = label_encoder.fit_transform(GAN_label_list)
n_classes = len(label_encoder.classes_)
print(n_classes)
print(image_labels_GAN)

# Convert the labels to one-hot encoded vectors
image_labels_GAN = to_categorical(image_labels_GAN, num_classes=n_classes)
print(image_labels_GAN[3000])

10
[0 0 0 ... 9 9 9]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [116]:
label_encoder = LabelEncoder()
image_labels_orignal = label_encoder.fit_transform(orignal_label_list)
n_classes = len(label_encoder.classes_)
print(n_classes)
print(image_labels_orignal)

# Convert the labels to one-hot encoded vectors
image_labels_orignal = to_categorical(image_labels_orignal, num_classes=n_classes)
print(image_labels_orignal[3000])

10
[0 0 0 ... 9 9 9]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [117]:
from sklearn.model_selection import train_test_split
print("making the training and testing split with 30% of the dataset as testing set")
GAN_x_train, GAN_x_test, GAN_y_train, GAN_y_test = train_test_split(normalized_image_list_GAN, image_labels_GAN, test_size=0.3, shuffle=True,random_state = 42)

orignal_x_train, orignal_x_test, orignal_y_train, orignal_y_test = train_test_split(normalized_image_list_orignal, image_labels_orignal, test_size=0.3, shuffle=True,random_state = 42)


making the training and testing split with 30% of the dataset as testing set


In [118]:
# label_gen = LabelBinarizer()
# image_labels = label_gen.fit_transform(enhanced_label_list)
# pickle.dump(label_gen,open('label_transform.pkl', 'wb'))
# n_classes = len(label_gen.classes_)
# print(n_classes)
# print(image_labels)

In [119]:
# from sklearn.model_selection import train_test_split
# print("making the training and testing split with 30% of the dataset as testing set")
# x_train, x_test, y_train, y_test = train_test_split(new_image_list, image_labels, test_size=0.3, shuffle=True,random_state = 42) 
# print(x_train.shape)
# print(x_test.shape)

In [120]:
model = Sequential()
model.add(Conv2D(filters = 32, padding = "same",activation = "relu",kernel_size=3, strides = 2,input_shape=(32,32,3)))
model.add(MaxPool2D(pool_size=(2,2),strides = 2))

model.add(Conv2D(filters = 32, padding = "same",activation = "relu",kernel_size=3))
model.add(MaxPool2D(pool_size=(2,2),strides = 2))

# model.add(Conv2D(filters = 32, padding = "same",activation = "relu",kernel_size=3))
# model.add(MaxPool2D(pool_size=(2,2),strides = 2))

model.add(Flatten())
model.add(Dense(128,activation="relu"))

#Output layer
model.add(Dense(10,kernel_regularizer=l2(0.01),activation = "linear"))
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 16, 16, 32)        896       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 8, 8, 32)          9248      
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 4, 4, 32)          0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 512)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)               65664     
_________________________________________________________________
dense_25 (Dense)             (None, 10)              

In [121]:
model.compile(optimizer="adam",loss="squared_hinge", metrics = ['accuracy'])

In [122]:
result = model.fit(GAN_x_train,GAN_y_train,epochs=15)

Epoch 1/15
219/219 [==============================] - 2s 5ms/step - loss: 0.4358 - accuracy: 0.2626
Epoch 2/15
219/219 [==============================] - 1s 5ms/step - loss: 0.2575 - accuracy: 0.6460
Epoch 3/15
219/219 [==============================] - 1s 5ms/step - loss: 0.1790 - accuracy: 0.8106
Epoch 4/15
219/219 [==============================] - 1s 5ms/step - loss: 0.1398 - accuracy: 0.8700
Epoch 5/15
219/219 [==============================] - 1s 5ms/step - loss: 0.1116 - accuracy: 0.9103
Epoch 6/15
219/219 [==============================] - 1s 5ms/step - loss: 0.0929 - accuracy: 0.9316
Epoch 7/15
219/219 [==============================] - 1s 5ms/step - loss: 0.0790 - accuracy: 0.9486
Epoch 8/15
219/219 [==============================] - 1s 6ms/step - loss: 0.0671 - accuracy: 0.9596
Epoch 9/15
219/219 [==============================] - 1s 6ms/step - loss: 0.0601 - accuracy: 0.9644
Epoch 10/15
219/219 [==============================] - 1s 6ms/step - loss: 0.0550 - accuracy: 0.9729

In [123]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

y_pred = model.predict(orignal_x_test)

y_pred_binary = np.where(y_pred > 0.5, 1, 0)
# y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate the true labels for the test set
y_true = orignal_y_test

accuracy = accuracy_score(orignal_y_test, y_pred_binary)
precision = precision_score(orignal_y_test, y_pred_binary, average='macro')
recall = recall_score(orignal_y_test, y_pred_binary, average='macro')
f1 = f1_score(orignal_y_test, y_pred_binary, average='macro')
# conf_matrix = confusion_matrix(y_test, y_pred_binary)


print("Accuracy:", accuracy * 100)
print("Precision:", precision * 100)
print("Recall:", recall * 100)
print("F1-score:", f1 * 100)

Accuracy: 11.166666666666666
Precision: 18.940036056853167
Recall: 10.985955435166158
F1-score: 7.1096126643397834


C:\Users\jagme\anaconda3\envs\new_env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# y_prob = model.predict(x_test)
# y_pred = np.where(y_prob[:len(x_test)] >= 0.5, 1, 0)

# # y_pred = model.predict(x_test)
# # print(y_pred)
# # # mse = mean_squared_error(y_test, y_pred)
# # # print(mse)
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)

# print("Accuracy:", accuracy * 100)
# print("Precision:", precision * 100)
# print("Recall:", recall * 100)
# print("F1-score:", f1 * 100)

Accuracy: 99.83333333333333
Precision: 100.0
Recall: 99.66101694915255
F1-score: 99.830220713073


In [17]:
model.save("plant_tomato_standAlone_DCGAN_svm.h5")